In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import xgboost as xgb
import copy
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')#忽略一些警告

## 读取数据

In [2]:
data=pd.read_csv("data/onehot_feature.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196499 entries, 0 to 196498
Data columns (total 40 columns):
时间           196499 non-null int64
小区名          196499 non-null int64
小区房屋出租数量     196499 non-null float64
楼层           196499 non-null int64
总楼层          196499 non-null float64
房屋面积         196499 non-null float64
房屋朝向         196499 non-null object
居住状态         196499 non-null float64
卧室数量         196499 non-null int64
厅的数量         196499 non-null int64
卫的数量         196499 non-null int64
出租方式         196499 non-null float64
区            196499 non-null float64
位置           196499 non-null float64
地铁线路         196499 non-null float64
地铁站点         196499 non-null float64
距离           196499 non-null float64
装修情况         196499 non-null float64
月租金          196499 non-null float64
log_rent     196499 non-null float64
新朝向          196499 non-null object
房+卫+厅        196499 non-null int64
房/总          196499 non-null float64
卫/总          196499 non-null float64
厅/总          1964

In [3]:
#将离散特征转换成字符串类型
colunms = ['时间', '新小区名', '居住状态', '出租方式', '区','位置','地铁线路','地铁站点','装修情况','户型','聚类特征']
for col in colunms:
    data[col] = data[col].astype(str)

## 获取特征集和目标值

In [4]:
data.columns

Index(['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积', '房屋朝向', '居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', '月租金',
       'log_rent', '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型',
       '平均值特征1', '小区平均值特征', '朝向平均值特征', '站点平均值特征', '位置平均值特征', '有地铁', '聚类特征',
       '平均值特征2', '小区线路数', '位置线路数', '新小区名', '小区条数大于100'],
      dtype='object')

In [5]:
x_columns=['时间', '新小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
       '平均值特征2','有地铁','小区线路数','位置线路数','小区条数大于100','小区平均值特征','朝向平均值特征',
           '站点平均值特征','位置平均值特征']
y_label='log_rent'
x=data[x_columns]
y=data[y_label]
y.isnull().sum()

0

## 构建训练函数

In [6]:
def feature_transformer(x,y,test_size=0.3,random_state=12):
    """
    负责分割数据集并转换onehot特征，返回转换后的稀疏矩阵和特征名
    """
    #1.重新命名列名
    #原列名
    cols=['时间', '新小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
       '平均值特征2','有地铁','小区线路数','位置线路数','小区条数大于100','小区平均值特征','朝向平均值特征',
           '站点平均值特征','位置平均值特征']
    #新列名
    new_cols=[chr(65+s)+str(i) for s in range(len(cols)//10+1) for i in range(10)]
    new_cols=new_cols[:len(cols)]
    #特征名映射字典
    cols_map={k:v for k,v in zip(cols,new_cols)}
    #重新命名列
    x.columns=[cols_map[k] for k in x.columns]
    
    #2.分割数据集
    train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=test_size,random_state=random_state)
    
    #3.转换onehot特征
    #返回稀疏矩阵，有两个优点：
    #1.占用内存大幅减小，让并行成为可能，不然并行的话，内存爆掉
    #2.可以加速xgboost训练
    vector=DictVectorizer(sparse=True)
    x_train=vector.fit_transform(train_x.to_dict(orient='records'))
    x_test=vector.transform(test_x.to_dict(orient='records'))
    features=vector.get_feature_names()
    
    #4.构建原始特征对应的新下标字典  
    #原始特征名
    
    #离散列特征下标字典
    feature_map={k:[] for k in cols}
    for i in range(len(features)):
        for col in cols:
            if features[i].startswith(cols_map[col]):#如果新特征以老特征开头
                feature_map[col].append(i)
                break
    return (x_train,x_test,train_y,test_y),feature_map,cols_map


def train(cols,data,feature_map,num_round = 500):
    """
    负责完成一次xgboost训练，返回测试集rmse
    """
    #1.获取数据集
    train_x,test_x,train_y,test_y=data
    #获取原始特征对应的新特征下标
    index=[]
    for col in cols:
        index.extend(feature_map[col])
    x_train=train_x[:,index]
    x_test=test_x[:,index]
    #3.构建数据格式
    #构建DMatrix数据，可以有效利用硬盘缓存，减少内存占用
    dtrain = xgb.DMatrix(x_train,train_y)
    dtest = xgb.DMatrix(x_test,test_y)
    
    #4.设置训练参数
    param = {'max_depth':5, 
             'eta':0.01, 
             'verbosity':1, 
             'objective':'reg:linear',
             'silent': 1,
             'gamma': 0.01,
             'min_child_weight': 1,
             
            }
    #5.模型训练
    bst = xgb.train(param, dtrain, num_round)
    
    #6.模型预测
    preds = bst.predict(dtest)
    preds=np.exp(preds)-1#转换成真实的租金
    y_true=np.exp(test_y)-1
    
    #7.模型评估
    return np.sqrt(mean_squared_error(y_true,preds))
    

In [7]:
#完成特征分割和转换
d,f_map,c_map=feature_transformer(x,y)

## 模型特征筛选

In [8]:
# 局部最优搜索
# 构造筛选特征函数
def select_features(cols,min_score):
    include_features=cols
    exclude_features=[]
    cols=np.array(cols)
    for i in range(cols.shape[0]):
        #选中的特征
        features=list(cols[list(set(range(cols.shape[0]))-set([i]))])
        print("开始第{}次训练:".format(i))
        print("未选中特征：",cols[i])
        rmse=train(features,d,f_map,2500)
        print("开始第{}次训练测试集成绩{}：".format(i,rmse))
        print("-"*10)
        if rmse<=min_score:
            exclude_features=cols[i]
            include_features=features
            min_score=rmse
    return min_score,include_features,exclude_features 

In [9]:
min_score=train(x_columns,d,f_map)

In [10]:
features=x_columns
min_score

2.631632039709192

In [11]:
i=1
while len(features)>0:
    print("开始第{}轮筛选:\n".format(i))
    ms,include_f,exclude_f=select_features(features,min_score)
    if len(exclude_f)<=0:
        break
    features=include_f
    min_score=ms
    print("第{}次筛选成绩:{}\n".format(i,min_score))
    i=i+1
    print("保留特征：",include_f)
    print("排除特征：",exclude_f)
    print("\n*************************\n")
    
print("最终特征：",features)
print("最好成绩：",min_score)

开始第1轮筛选:

开始第0次训练:
未选中特征： 时间
开始第0次训练测试集成绩2.3461997068545446：
----------
开始第1次训练:
未选中特征： 新小区名
开始第1次训练测试集成绩2.3358563620495256：
----------
开始第2次训练:
未选中特征： 小区房屋出租数量
开始第2次训练测试集成绩2.36631512364606：
----------
开始第3次训练:
未选中特征： 楼层


KeyboardInterrupt: 

In [12]:
cols=['小区房屋出租数量','房屋面积', '居住状态', '出租方式', '位置', '地铁站点', '距离', '装修情况', 
      '新朝向', '房+卫+厅', '房/总', '卫/总',  '卧室面积', '楼层比', '平均值特征1', '平均值特征2', 
      '小区线路数', '位置线路数','小区条数大于100']
train(cols,d,f_map)

2.6888532158953726

## 参数搜索
### 构建交叉验证和参数搜索函数

In [13]:
from sklearn.model_selection import KFold
#构建交叉验证函数
def train_cv(data,target,params,num_round=200,k_fold=5,silent=0):
    """
    负责完成一种参数组合的情况下k_flod折交叉验证的平均rmse值
    """
    rmses=[]
    #数据分割
    kfold= KFold(n_splits=k_fold,random_state =None,shuffle=True)
    for i,(train_index,val_index) in zip(range(k_fold),kfold.split(data,target)):
        train_x,val_x,train_y,val_y=data[train_index,:],data[val_index,:],target[train_index],target[val_index]
        #构建DMatrix数据
        dtrain = xgb.DMatrix(train_x,train_y)
        dtest = xgb.DMatrix(val_x,val_y)
        if silent==0:
            print("开始第{}/{}折验证：".format(i,k_fold))
        #模型训练
        bst = xgb.train(params, dtrain, num_round)
        
        #6.模型预测
        preds = bst.predict(dtest)
        preds=np.exp(preds)-1#转换成真实的租金
        y_true=np.exp(val_y)-1
        rmse=np.sqrt(mean_squared_error(y_true,preds))
        if silent==0:
            print("第{}/{}折验证rmse：{}".format(i,k_fold,rmse))
        rmses.append(rmse)
    return sum(rmses)/k_fold

def search_params(x,y,params_grid,n_estimators=200,cv=3,silent=0):
    min_rmse=9999
    best_params=None
    params_list=[[]]
    
    #根据参数表格构建所有参数组合
    for k,v in params_grid.items():
        if isinstance(v,list):
            temp=params_list
            params_list=[i+[j] for j in v for i in temp]
        else:
            params_list=[i+[v] for i in params_list]
    params_list=[{k:v for k,v in zip(params_grid.keys(),v_list)} for v_list in params_list]
    
    for i,params in zip(range(len(params_list)),params_list):
        if silent==0:
            print("开始实验第{}组参数：".format(i),params)
        rmse=train_cv(data=x,target=y,params=params,num_round=n_estimators,k_fold=cv)
        if silent==0:
            print("第{}组参数平均rmse：{}".format(i,rmse))
            print("-"*50)
        if rmse<min_rmse:
            min_rmse=rmse
            best_params=params
    return min_rmse,best_params
              

## 开始搜索

In [14]:
params_dict={
    "objective":'reg:linear',
    'eta':[0.01,0.1,0.5],
    'gamma': [0.01,0.05,0.1],
    'silent': 1,
    'max_depth':[15,25,35], 
    'min_child_weight':[0.5,1,3],
}
cols=['小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '卫的数量',  '位置',  '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
       '平均值特征2','有地铁','小区线路数','位置线路数','小区条数大于100','小区平均值特征','朝向平均值特征',
           '站点平均值特征','位置平均值特征']
#1.获取数据集
train_x,test_x,train_y,test_y=d
#获取原始特征对应的新特征下标
index=[]
for col in cols:
    index.extend(f_map[col])
x_train=train_x[:50000,index]#只用前50000条数据做运算
x_test=test_x[:,index]
#由于要用新下标访问，所以要重置索引
train_y=train_y.reset_index(drop=True)[:50000]
test_y=test_y.reset_index(drop=True)
search_params(x=x_train,y=train_y,params_grid=params_dict,n_estimators=1000)

开始实验第0组参数： {'objective': 'reg:linear', 'eta': 0.01, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.3334754684541483
开始第1/3折验证：
第1/3折验证rmse：2.0875718319952443
开始第2/3折验证：
第2/3折验证rmse：2.171265541692766
第0组参数平均rmse：2.1974376140473866
--------------------------------------------------
开始实验第1组参数： {'objective': 'reg:linear', 'eta': 0.1, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.243130033305063
开始第1/3折验证：
第1/3折验证rmse：2.2967056571739377
开始第2/3折验证：
第2/3折验证rmse：2.2360509519219645
第1组参数平均rmse：2.2586288808003214
--------------------------------------------------
开始实验第2组参数： {'objective': 'reg:linear', 'eta': 0.5, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.4488787219374224
开始第1/3折验证：
第1/3折验证rmse：2.3035791735080684
开始第2/3折验证：
第2/3折验证rmse：2.333795157155841
第2组参数平均rmse：2.3620843508671108
--------------------------------------------------
开始实验第3组参数： {'ob

KeyboardInterrupt: 

In [15]:
# 全数据集训练
#1.获取数据集
train_x,test_x,train_y,test_y=d
#获取原始特征对应的新特征下标
index=[]
for col in cols:
    index.extend(f_map[col])
x_train=train_x[:,index]
x_test=test_x[:,index]
#由于要用新下标访问，所以要重置索引
train_y=train_y.reset_index(drop=True)
test_y=test_y.reset_index(drop=True)

## 利用搜索的参数训练模型

In [19]:
# sklearn接口
# xgb_model = xgb.XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=500,verbosity=1, objective='reg:linear',random_state=12,)
# xgb_model.fit(new_train_x, train_y, early_stopping_rounds=10, eval_metric="rmse",
#         eval_set=[(new_test_x, test_y)])


# 原生接口
params={
    "objective":'reg:linear',
    'eta':0.01,
    'gamma': 0.05,
    'silent': 1,
    'max_depth':25, 
    'min_child_weight':0.5,
    'sub_sample':0.6,
    'reg_alpha':0.5,
    'reg_lambda':0.8,
    'colsample_bytree':0.5
}
dtrain = xgb.DMatrix(x_train,train_y)
dtest = xgb.DMatrix(x_test,test_y)
# bst = xgb.train(params, dtrain, num_boost_round=1500)
bst = xgb.train(params, dtrain, num_boost_round=500)
        
#6.模型预测
preds = bst.predict(dtest)
preds=np.exp(preds)-1#转换成真实的租金
y_true=np.exp(test_y)-1
rmse=np.sqrt(mean_squared_error(y_true,preds))
rmse

1.6782888625446752

In [20]:
# 网格搜索和交叉验证
cv_params = {'n_estimators': [400, 500, 600, 700, 800]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1)
optimized_GBM.fit(x_train, train_y)
evalute_result = optimized_GBM.grid_scores_
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 46.5min finished


AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'